### Importing Libraries

In [1]:
import pandas as pd 
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import time

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

### Create Initial Table

#### Request from the list of United States cities by population

In [2]:
link = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
response = requests.get(link)
if response.status_code == 200:
    results_page = BeautifulSoup(response.content,'lxml')
    table = results_page.find_all('table',"wikitable sortable")[0]
    columns = [r.get_text()[:-1] for r in table.find_all('th')]
    table_body = table.find_all('tr')[1:]
    df_table = [[td.get_text()[:-1] for td in tr.find_all('td') if '\xa0mi' not in td.get_text()] for tr in table_body]
    data = pd.DataFrame(df_table, columns = columns)
    data.head()

#### Cleaning the initial table

In [3]:
columns = ['2018_rank','City','State','2018_estimate','2010_census','Change (%)','2016_land_area (km2)',
           '2016_population_density (/km2)','Location']
data.columns = columns
data['City'] = data['City'].apply(lambda x: x.split('[')[0])
data['State'] = data['State'].apply(lambda x: x.replace('\xa0',''))
data['2018_estimate'] = data['2018_estimate'].apply(lambda x: int(x.replace(',','')))
data['2010_census'] = data['2010_census'].apply(lambda x: int(x.replace(',','')))
data.at[279,'Change (%)'] = '00'
data['Change (%)'] = data['Change (%)'].apply(lambda x : float(x[:-1].replace('−','-'))) 
data['2016_land_area (km2)'] = data['2016_land_area (km2)'].apply( lambda x: float(x.split("\xa0")[0].replace(',','')))
data['2016_population_density (/km2)'] = data['2016_population_density (/km2)'].apply( lambda x: int(x.split('/')[0].replace(',',
                                                                                                                             '')))
data['Location'] = data['Location'].apply( lambda x: x.split('/')[0].split('\ufeff')[0])

In [4]:
data.head()

,2018_rank,City,State,2018_estimate,2010_census,Change (%),2016_land_area (km2),2016_population_density (/km2),Location
0,1,New York City,New York,8398748,8175133,2.74,780.9,10933,40°39′49″N 73°56′19″W
1,2,Los Angeles,California,3990456,3792621,5.22,1213.9,3276,34°01′10″N 118°24′39″W
2,3,Chicago,Illinois,2705994,2695598,0.39,588.7,4600,41°50′15″N 87°40′54″W
3,4,Houston,Texas,2325502,2100263,10.72,1651.1,1395,29°47′12″N 95°23′27″W
4,5,Phoenix,Arizona,1660272,1445632,14.85,1340.6,1200,33°34′20″N 112°05′24″W


### Complete data with individual city pages

In [5]:
base_url = 'https://en.wikipedia.org/wiki/'
cities = data['City'].values

#### Initiate the new database

In [6]:
descriptions = []
nicknames = []
regions = []
counties = []
govern_types = []
govern_bodies = []
govern_mayors = []
area_lands = []
area_waters = []
area_metros = []
elevations = []
CSA = []
time_zones = []
ZIP_codes = []
area_codes = []
FIPS_codes = []
airports = []
websites = []
features = {'Description':descriptions, 'Nickname':nicknames, 'Region':regions, 'County':counties, 'Govern_type':govern_types,
            'Govern_body':govern_bodies, 'Govern_mayor':govern_mayors, 'Area_land':area_lands, 'Area_water':area_waters,
           'Area_metro':area_metros, 'Elevation':elevations, 'CSA':CSA, 'Time_zone':time_zones, 'ZIP_code':ZIP_codes,
            'Area_code':area_codes, 'FIPS_code':FIPS_codes, 'Airport':airports, 'Website':websites}

#### Function to clean the city's description

In [7]:
def clean_description(s):
    s = re.sub(r'\[.\]|\[..\]','',s)
    s = s.strip('\n')
    s = s.strip('\xa0')
    return(s)

#### Loop over the 314 cities to gather the new information

In [8]:
for i in range(len(cities)):
    while True:
        try:
            response = requests.get(base_url+cities[i].replace(' ','_')+',_'+data.iloc[i]['State'].replace(' ','_'))
            break
        except:
            time.sleep(3)
    print(cities[i], '-----','%d/%d'%(i+1,len(cities)))
    if response.status_code == 200:
        results_page = BeautifulSoup(response.content,'lxml')
        k = 0
        while True:
            try:
                results_page.find_all('p')[k].attrs['class'][0]
                k += 1 
            except:
                descriptions.append(clean_description(results_page.find_all('p')[k].get_text()))
                break

        for tr in results_page.find('tbody').find_all('tr'):
            try:
                if 'Nickname' in tr.find('td').get_text():
                    nicknames.append(tr.find('td').find('div').get_text())
            except:
                pass
            try: 
                if 'Region' in tr.find('th').get_text():
                    regions.append(tr.find('td').get_text())
            except:
                pass
            try: 
                if 'County' in tr.find('th').get_text() or 'ounties' in tr.find('th').get_text():
                    counties.append(tr.find('td').get_text())
            except:
                pass
            try: 
                if 'Type' in tr.find('th').get_text():
                    govern_types.append(tr.find('td').get_text())
            except:
                pass
            try: 
                if 'Body' in tr.find('th').get_text():
                    govern_bodies.append(tr.find('td').get_text())
            except:
                pass
            try: 
                if 'Mayor' in tr.find('th').get_text():
                    govern_mayors.append(tr.find('td').get_text())
            except:
                pass
            try: 
                if 'Land' in tr.find('th').get_text():
                    area_lands.append(tr.find('td').get_text())
            except:
                pass
            try: 
                if 'Water' in tr.find('th').get_text():
                    area_waters.append(tr.find('td').get_text())
            except:
                pass
            try: 
                if 'Metro' in tr.find('th').get_text() and 'GMP' not in tr.find('th').get_text():
                    area_metros.append(tr.find('td').get_text())
            except:
                pass
            try: 
                if 'Elevation' in tr.find('th').get_text():
                    elevations.append(tr.find('td').get_text())
            except:
                pass
            try: 
                if 'CSA' in tr.find('th').get_text():
                    if tr.find('td').get_text()[0].isdigit():
                        CSA.append(tr.find('td').get_text())
            except:
                pass
            try: 
                if 'Time zone' in tr.find('th').get_text():
                    time_zones.append(tr.find('td').get_text())
            except:
                pass
            try: 
                if 'ZIP' in tr.find('th').get_text():
                    ZIP_codes.append(tr.find('td').get_text())
            except:
                pass
            try: 
                if 'Area code' in tr.find('th').get_text():
                    area_codes.append(tr.find('td').get_text())
            except:
                pass
            try: 
                if 'FIPS' in tr.find('th').get_text():
                    FIPS_codes.append(tr.find('td').get_text())
            except:
                pass
            try: 
                if 'irport' in tr.find('th').get_text():
                    airports.append(tr.find('td').get_text())
            except:
                pass
            try: 
                if 'Website' in tr.find('th').get_text():
                    websites.append(tr.find('td').get_text())
            except:
                pass
    for f in list(features.keys()):
        if len(features[f]) < i+1:
            features[f].append('')
        while len(features[f]) > i+1:
            features[f].pop()

New York City ----- 1/314
Los Angeles ----- 2/314
Chicago ----- 3/314
Houston ----- 4/314
Phoenix ----- 5/314
Philadelphia ----- 6/314
San Antonio ----- 7/314
San Diego ----- 8/314
Dallas ----- 9/314
San Jose ----- 10/314
Austin ----- 11/314
Jacksonville ----- 12/314
Fort Worth ----- 13/314
Columbus ----- 14/314
San Francisco ----- 15/314
Charlotte ----- 16/314
Indianapolis ----- 17/314
Seattle ----- 18/314
Denver ----- 19/314
Washington, D.C. ----- 20/314
Boston ----- 21/314
El Paso ----- 22/314
Detroit ----- 23/314
Nashville ----- 24/314
Portland ----- 25/314
Memphis ----- 26/314
Oklahoma City ----- 27/314
Las Vegas ----- 28/314
Louisville ----- 29/314
Baltimore ----- 30/314
Milwaukee ----- 31/314
Albuquerque ----- 32/314
Tucson ----- 33/314
Fresno ----- 34/314
Mesa ----- 35/314
Sacramento ----- 36/314
Atlanta ----- 37/314
Kansas City ----- 38/314
Colorado Springs ----- 39/314
Miami ----- 40/314
Raleigh ----- 41/314
Omaha ----- 42/314
Long Beach ----- 43/314
Virginia Beach ----- 44/3

In [9]:
data2 = pd.DataFrame(features)

In [10]:
data2.head()

,Description,Nickname,Region,County,Govern_type,Govern_body,Govern_mayor,Area_land,Area_water,Area_metro,Elevation,CSA,Time_zone,ZIP_code,Area_code,FIPS_code,Airport,Website
0,"The City of New York, usually called either Ne...",See Nicknames of New York City,Mid-Atlantic,Bronx (The Bronx)Kings (Brooklyn)New York (Man...,Mayor–Council,New York City Council,Bill de Blasio (D),302.643 sq mi (783.84 km2),165.841 sq mi (429.53 km2),"13,318 sq mi (34,490 km2)",33 ft (10 m),"22,679,948 [5] (1st)",UTC−05:00 (EST),"100xx–104xx, 11004–05, 111xx–114xx, 116xx","212/646/332, 718/347/929, 917",36-51000,"John F. Kennedy International Airport, Newark ...",NYC.gov
1,Los Angeles (/lɔːs ˈændʒələs/ (listen); Spanis...,"L.A., City of Angels,[1] The Entertainment Cap...",,Los Angeles,Mayor-Council-Commission[5],Los Angeles City Council,Eric Garcetti (D)[6],"468.74 sq mi (1,214.03 km2)",34.02 sq mi (88.12 km2) 6.7%,"13,131,431 (U.S.: 2nd)",305 ft (93 m),"18,679,763 (U.S.: 2nd)",UTC−08:00 (Pacific),"\nList\n90001–90084, 90086–90089, 90091, 90093...","213/323, 310/424, 747/818",06-44000,,Official website
2,"Chicago (/ʃɪˈkɑːɡoʊ/ (listen), locally also /ʃ...","Windy City, Chi-Town, City of Big Shoulders,[1...",,"Cook, DuPage",Mayor–council,Chicago City Council,Lori Lightfoot (D),227.34 sq mi (588 km2),6.80 sq mi (17.62 km2) 3.0%,"10,874 sq mi (28,160 km2)",594 ft (181 m),"9,901,711 (US: 3rd)[4]",UTC−06:00 (Central),"606xx, 607xx, 608xx",312/872 and 773/872,17-14000,"Chicago O'Hare, Chicago Midway",www.cityofchicago.org
3,Houston (/ˈhjuːstən/ (listen) HEW-stən) is the...,Space City (official) more ...,,"Harris, Fort Bend, Montgomery",Mayor–council,Houston City Council,Sylvester Turner (D),"599.59 sq mi (1,552.9 km2)",,"1,062 sq mi (2,750 km2)",80 ft (32 m),,UTC−6 (CST),"770xx, 772xx (P.O. Boxes)","713, 281, 832, 346",48-35000[4],"George Bush Intercontinental Airport (IAH), Wi...",houstontx.gov
4,Phoenix (/ˈfiːnɪks/) is the capital and most p...,"""Valley of the Sun"", ""The Valley""",,Maricopa,Council-Manager,Phoenix City Council,Kate Gallego,"517.64 sq mi (1,340.69 km2)",1.25 sq mi (3.25 km2),"14,565.76 sq mi (37,725.1 km2)","1,086 ft (331 m)",,UTC−7 (MST (no DST)),85001–85099,\nEast: 480\nCentral: 602\nWest: 623\n,04-55000,Phoenix Sky Harbor International Airport,www.phoenix.gov


#### Cleaning of the new database

Not enough values for Regions

In [11]:
data2.drop(columns = 'Region', inplace = True)

Cleaning Functions

In [12]:
def get_csa(s):
    try:
        return int(s.split(' ')[0].replace(',',''))
    except:
        return np.nan

In [13]:
def get_km2(s):
    try:
        return float(s.split('(')[-1].split('\xa0')[0].replace(',',''))
    except:
        return np.nan

In [14]:
def clean_text(t):
    t = re.sub(r'\[.\]|\[..\]','',t)
    return t

The cleaning process is different for every columns

In [15]:
data2['Nickname'] = data2['Nickname'].apply(lambda x:re.sub(r'"|\([^)]*\)',''
                                                            ,clean_text(x.split('See Nicknames')[0])).replace(';',',').strip())
data2['County'] = data2['County'].apply(lambda x: clean_text(x).strip('\n').strip())
data2['Govern_type'] = data2['Govern_type'].apply(lambda x: clean_text(x).strip())
data2['Govern_body'] =  data2['Govern_body'].apply(lambda x: clean_text(x).strip())
data2['Govern_mayor'] =  data2['Govern_mayor'].apply(lambda x: clean_text(x).strip())
data2['Area_land (km2)'] = data2['Area_land'].apply(lambda x: get_km2(x))
data2['Area_water (km2)'] = data2['Area_water'].apply(lambda x: get_km2(x))
data2['Area_metro (km2)'] = data2['Area_metro'].apply(lambda x: get_km2(x))
data2['Elevation (m)'] = data2['Elevation'].apply(lambda x: get_km2(x))
data2['CSA'] = data2['CSA'].apply(lambda x: get_csa(x))
data2['ZIP_code']=data2['ZIP_code'].apply(lambda x: re.sub(r'[^-,x0-9 ]*|[0-9][0-9] ',''
                                                           ,clean_text(x).replace('–','-').strip('\n')).strip())
data2['ZIP_code']=data2['ZIP_code'].apply(lambda x: re.sub(r'[0-9][0-9] ','',x))
data2['Area_code']=data2['Area_code'].apply(lambda x: re.sub(r'[^-,0-9/ ]*','',clean_text(x)).
                                            replace('  ',' ').replace('/','-').replace(', ',',').replace(' ',',').strip())
data2['FIPS_code']=data2['FIPS_code'].apply(lambda x:clean_text(x).strip())

In [29]:
data2.head()

,Description,Nickname,County,Govern_type,Govern_body,Govern_mayor,Area_land,Area_water,Area_metro,Elevation,CSA,Time_zone,ZIP_code,Area_code,FIPS_code,Airport,Website,Area_land (km2),Area_water (km2),Area_metro (km2),Elevation (m)
0,"The City of New York, usually called either Ne...",,Bronx (The Bronx)Kings (Brooklyn)New York (Man...,Mayor–Council,New York City Council,Bill de Blasio (D),302.643 sq mi (783.84 km2),165.841 sq mi (429.53 km2),"13,318 sq mi (34,490 km2)",33 ft (10 m),22679948.0,UTC−05:00 (EST),"100xx-104xx, 11004-05, 111xx-114xx, 116xx","212-646-332,718-347-929,917",36-51000,"John F. Kennedy International Airport, Newark ...",NYC.gov,783.84,429.53,34490.0,10.0
1,Los Angeles (/lɔːs ˈændʒələs/ (listen); Spanis...,"L.A., City of Angels, The Entertainment Capita...",Los Angeles,Mayor-Council-Commission,Los Angeles City Council,Eric Garcetti (D),"468.74 sq mi (1,214.03 km2)",34.02 sq mi (88.12 km2) 6.7%,"13,131,431 (U.S.: 2nd)",305 ft (93 m),18679763.0,UTC−08:00 (Pacific),"90001-90084, 90086-90089, 90091, 90093-90097, ...","213-323,310-424,747-818",06-44000,,Official website,1214.03,88.12,NaN,93.0
2,"Chicago (/ʃɪˈkɑːɡoʊ/ (listen), locally also /ʃ...","Windy City, Chi-Town, City of Big Shoulders, S...","Cook, DuPage",Mayor–council,Chicago City Council,Lori Lightfoot (D),227.34 sq mi (588 km2),6.80 sq mi (17.62 km2) 3.0%,"10,874 sq mi (28,160 km2)",594 ft (181 m),9901711.0,UTC−06:00 (Central),"606xx, 607xx, 608xx","312-872,773-872",17-14000,"Chicago O'Hare, Chicago Midway",www.cityofchicago.org,588.00,17.62,28160.0,181.0
3,Houston (/ˈhjuːstən/ (listen) HEW-stən) is the...,Space City more ...,"Harris, Fort Bend, Montgomery",Mayor–council,Houston City Council,Sylvester Turner (D),"599.59 sq mi (1,552.9 km2)",,"1,062 sq mi (2,750 km2)",80 ft (32 m),NaN,UTC−6 (CST),"770xx, 772xx x","713,281,832,346",48-35000,"George Bush Intercontinental Airport (IAH), Wi...",houstontx.gov,1552.90,NaN,2750.0,32.0
4,Phoenix (/ˈfiːnɪks/) is the capital and most p...,"Valley of the Sun, The Valley",Maricopa,Council-Manager,Phoenix City Council,Kate Gallego,"517.64 sq mi (1,340.69 km2)",1.25 sq mi (3.25 km2),"14,565.76 sq mi (37,725.1 km2)","1,086 ft (331 m)",NaN,UTC−7 (MST (no DST)),85001-85099,",480,602,623",04-55000,Phoenix Sky Harbor International Airport,www.phoenix.gov,1340.69,3.25,37725.1,331.0


### Merge the two dataset

In [30]:
data3 = pd.concat([data,data2],axis = 1)

In [31]:
data3.head()

,2018_rank,City,State,2018_estimate,2010_census,Change (%),2016_land_area (km2),2016_population_density (/km2),Location,Description,Nickname,County,Govern_type,Govern_body,Govern_mayor,Area_land,Area_water,Area_metro,Elevation,CSA,Time_zone,ZIP_code,Area_code,FIPS_code,Airport,Website,Area_land (km2),Area_water (km2),Area_metro (km2),Elevation (m)
0,1,New York City,New York,8398748,8175133,2.74,780.9,10933,40°39′49″N 73°56′19″W,"The City of New York, usually called either Ne...",,Bronx (The Bronx)Kings (Brooklyn)New York (Man...,Mayor–Council,New York City Council,Bill de Blasio (D),302.643 sq mi (783.84 km2),165.841 sq mi (429.53 km2),"13,318 sq mi (34,490 km2)",33 ft (10 m),22679948.0,UTC−05:00 (EST),"100xx-104xx, 11004-05, 111xx-114xx, 116xx","212-646-332,718-347-929,917",36-51000,"John F. Kennedy International Airport, Newark ...",NYC.gov,783.84,429.53,34490.0,10.0
1,2,Los Angeles,California,3990456,3792621,5.22,1213.9,3276,34°01′10″N 118°24′39″W,Los Angeles (/lɔːs ˈændʒələs/ (listen); Spanis...,"L.A., City of Angels, The Entertainment Capita...",Los Angeles,Mayor-Council-Commission,Los Angeles City Council,Eric Garcetti (D),"468.74 sq mi (1,214.03 km2)",34.02 sq mi (88.12 km2) 6.7%,"13,131,431 (U.S.: 2nd)",305 ft (93 m),18679763.0,UTC−08:00 (Pacific),"90001-90084, 90086-90089, 90091, 90093-90097, ...","213-323,310-424,747-818",06-44000,,Official website,1214.03,88.12,NaN,93.0
2,3,Chicago,Illinois,2705994,2695598,0.39,588.7,4600,41°50′15″N 87°40′54″W,"Chicago (/ʃɪˈkɑːɡoʊ/ (listen), locally also /ʃ...","Windy City, Chi-Town, City of Big Shoulders, S...","Cook, DuPage",Mayor–council,Chicago City Council,Lori Lightfoot (D),227.34 sq mi (588 km2),6.80 sq mi (17.62 km2) 3.0%,"10,874 sq mi (28,160 km2)",594 ft (181 m),9901711.0,UTC−06:00 (Central),"606xx, 607xx, 608xx","312-872,773-872",17-14000,"Chicago O'Hare, Chicago Midway",www.cityofchicago.org,588.00,17.62,28160.0,181.0
3,4,Houston,Texas,2325502,2100263,10.72,1651.1,1395,29°47′12″N 95°23′27″W,Houston (/ˈhjuːstən/ (listen) HEW-stən) is the...,Space City more ...,"Harris, Fort Bend, Montgomery",Mayor–council,Houston City Council,Sylvester Turner (D),"599.59 sq mi (1,552.9 km2)",,"1,062 sq mi (2,750 km2)",80 ft (32 m),NaN,UTC−6 (CST),"770xx, 772xx x","713,281,832,346",48-35000,"George Bush Intercontinental Airport (IAH), Wi...",houstontx.gov,1552.90,NaN,2750.0,32.0
4,5,Phoenix,Arizona,1660272,1445632,14.85,1340.6,1200,33°34′20″N 112°05′24″W,Phoenix (/ˈfiːnɪks/) is the capital and most p...,"Valley of the Sun, The Valley",Maricopa,Council-Manager,Phoenix City Council,Kate Gallego,"517.64 sq mi (1,340.69 km2)",1.25 sq mi (3.25 km2),"14,565.76 sq mi (37,725.1 km2)","1,086 ft (331 m)",NaN,UTC−7 (MST (no DST)),85001-85099,",480,602,623",04-55000,Phoenix Sky Harbor International Airport,www.phoenix.gov,1340.69,3.25,37725.1,331.0


In [34]:
data3.to_csv('./data/us_cities.csv',index = False)